In [88]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import talib  
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

tickers = ['META', 'AAPL', 'AMZN', 'NFLX', 'GOOGL', 'SPY']#FAANG portfolio with SPY for market proxy

In [89]:
class DataPreprocessor:
    def __init__(self, tickers,):
        self.tickers = tickers
        self.data = None
        
    def run_pipeline(self, start_date='2015-01-01', end_date='2024-12-31'):
        self._download_data(start_date, end_date)
        self._clean_data()
        self._validate_data()
        return self.data
    
    def _download_data(self, start_date, end_date):
        all_data = []
        
        for ticker in self.tickers:
            
            # Download with adjusted close prices and make column names lower
            df = yf.download(ticker, start=start_date, end=end_date, progress=False, auto_adjust=True)
            
            if isinstance(df.columns, pd.MultiIndex): 
                df.columns = [col[0].lower() for col in df.columns]
            else:
                df.columns = [col.lower() for col in df.columns]
            
            df.index.names = [name.lower() if name else 'date' for name in df.index.names]
            
            df = df[['open', 'high', 'low', 'close', 'volume']] # Keep only essential columns
            
            df['ticker'] = ticker # Add ticker column and set multi-index, organize data
            df = df.reset_index()
            df.set_index(['ticker', 'date'], inplace=True)
            
            all_data.append(df)
        
        self.data = pd.concat(all_data, axis=0).sort_index()

    def _clean_data(self):
        if self.data is None:
            raise ValueError("No data to clean!")
        
        # 1. Fill small gaps (forward fill then backward fill)
        self.data = self.data.groupby(level=0, group_keys=False).apply(lambda x: x.ffill().bfill())
        
        # 2. Drop any remaining NaN (usually at the beginning)
        self.data = self.data.dropna()

    def _validate_data(self):
        # Check date alignment
        date_counts = {}
        for ticker in self.data.index.get_level_values(0).unique():
            dates = self.data.xs(ticker, level=0).index
            date_counts[ticker] = len(dates)
        
        if len(set(date_counts.values())) == 1:
            print(f"All tickers have {list(date_counts.values())[0]} trading days")
        else:
            print("tickers have different dates")
        
        # Quick statistical check
        sample_ticker = self.data.index.get_level_values(0).unique()[0]
        print(f"\n📊 Sample statistics ({sample_ticker} close prices):")
        sample_close = self.data.xs(sample_ticker, level=0)['close']
        print(f"    • Mean: ${sample_close.mean():.2f}")
        print(f"    • Max:  ${sample_close.max():.2f}")

# Create an instance and run the pipeline
preprocessor = DataPreprocessor(tickers)
data = preprocessor.run_pipeline(start_date='2015-01-01', end_date='2024-12-31')

# Preview the data
print(data.shape)
print(data.index.get_level_values('ticker').unique().tolist())
print(data.index.get_level_values('date').min(), "to", data.index.get_level_values('date').max())
data.head(10)

All tickers have 2515 trading days

📊 Sample statistics (AAPL close prices):
    • Mean: $93.80
    • Max:  $257.85
(15090, 5)
['AAPL', 'AMZN', 'GOOGL', 'META', 'NFLX', 'SPY']
2015-01-02 00:00:00 to 2024-12-30 00:00:00


open       high        low      close     volume
ticker date                                                             
AAPL   2015-01-02  24.694237  24.705322  23.798602  24.237553  212818400
       2015-01-05  24.006988  24.086797  23.368517  23.554737  257142000
       2015-01-06  23.619027  23.816332  23.195595  23.556953  263188400
       2015-01-07  23.765350  23.987042  23.654504  23.887281  160423600
       2015-01-08  24.215381  24.862721  24.097883  24.805080  237458000
       2015-01-09  24.978000  25.106581  24.432638  24.831684  214798000
       2015-01-12  24.962475  24.969126  24.120048  24.219809  198603200
       2015-01-13  24.703103  25.006821  24.144441  24.434856  268367600
       2015-01-14  24.173261  24.494713  24.053547  24.341747  195826400
       2015-01-15  24.386086  24.399387  23.645637  23.681107  240056000

In [90]:
class FeatureEngineer:
    def __init__(self, data):
        self.data = data.copy()
        self.finished_features = []
        
    def run_pipeline(self):
        self.core_price_features()
        self.math_rule_features()
        self.momentum_features()
        self.volatility_features()
        self.volume_features()
        self.lagged_features()
        self.target_variable()
        return self.data
    
    def _apply_by_ticker(self, func):
        return self.data.groupby(level='ticker', group_keys=False).apply(func)
    
    def core_price_features(self):
        def calc(df):
            close, high, low, open_ = df['close'], df['high'], df['low'], df['open']
            
            # Returns
            df['log_return'] = np.log(close / close.shift(1))
            df['overnight_return'] = np.log(open_ / close.shift(1))
            df['intraday_return'] = np.log(close / open_)
            # Volatility
            df['volatility_20d'] = df['log_return'].rolling(20).std() * np.sqrt(252)
            df['atr_14'] = talib.ATR(high, low, close, timeperiod=14)
            # SMAs
            sma10 = talib.SMA(close, timeperiod=10)
            sma20 = talib.SMA(close, timeperiod=20)
            sma50 = talib.SMA(close, timeperiod=50)
            #ratios
            df['price_sma20_ratio'] = close / sma20
            df['price_sma50_ratio'] = close / sma50
            df['sma10_sma20_ratio'] = sma10 / sma20
            return df
        
        self.data = self._apply_by_ticker(calc)
        self.finished_features += ['log_return', 'overnight_return', 'intraday_return', 
                            'volatility_20d', 'atr_14', 'price_sma20_ratio', 
                            'price_sma50_ratio', 'sma10_sma20_ratio']
    
    def math_rule_features(self):
        """15 binary features for rule-based model"""
        def calc(df):
            close, high, low, volume = df['close'], df['high'], df['low'], df['volume']
            
            # SMAs
            sma10 = talib.SMA(close, timeperiod=10)
            sma20 = talib.SMA(close, timeperiod=20)
            sma50 = talib.SMA(close, timeperiod=50)
            sma200 = talib.SMA(close, timeperiod=200)
            
            # Trends
            df['golden_cross'] = (sma50 > sma200).astype(int)
            df['short_uptrend'] = (sma10 > sma20).astype(int)
            df['price_above_sma20'] = (close > sma20).astype(int)
            df['price_above_sma50'] = (close > sma50).astype(int)
            
            # Momentum
            rsi = talib.RSI(close, timeperiod=14)
            macd, signal, _ = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
            stoch_k, _ = talib.STOCH(high, low, close, fastk_period=14, slowk_period=3, slowd_period=3)
            roc = talib.ROC(close, timeperiod=10)
            
            df['rsi_oversold'] = (rsi < 30).astype(int)
            df['rsi_overbought'] = (rsi > 70).astype(int)
            df['macd_bullish'] = (macd > signal).astype(int)
            df['roc_positive'] = (roc > 0).astype(int)
            df['stoch_oversold'] = (stoch_k < 20).astype(int)
            
            # Volatility/Reversion 
            upper, _, lower = talib.BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2)
            bb_pos = (close - lower) / (upper - lower)
            vol_20d = df['log_return'].rolling(20).std() * np.sqrt(252)
            vol_75pct = vol_20d.expanding().quantile(0.75)
            
            df['bb_oversold'] = (bb_pos < 0.2).astype(int)
            df['bb_overbought'] = (bb_pos > 0.8).astype(int)
            df['high_volatility'] = (vol_20d > vol_75pct).astype(int)
            
            # Volume 
            vol_sma20 = talib.SMA(volume, timeperiod=20)
            vol_ratio = volume / vol_sma20
            price_up = close > close.shift(1)
            price_down = close < close.shift(1)
            
            df['volume_spike'] = (vol_ratio > 1.5).astype(int)
            df['volume_confirmation'] = (price_up & (vol_ratio > 1)).astype(int)
            df['volume_divergence'] = (price_down & (vol_ratio > 1)).astype(int)
            return df
        
        self.data = self._apply_by_ticker(calc)
        self.finished_features += ['golden_cross', 'short_uptrend', 'price_above_sma20', 'price_above_sma50',
                            'rsi_oversold', 'rsi_overbought', 'macd_bullish', 'roc_positive', 'stoch_oversold',
                            'bb_oversold', 'bb_overbought', 'high_volatility',
                            'volume_spike', 'volume_confirmation', 'volume_divergence']
    
    def momentum_features(self):
        """5 momentum oscillators"""
        def calc(df):
            close, high, low = df['close'], df['high'], df['low']
            
            df['rsi_14'] = talib.RSI(close, timeperiod=14)
            macd, signal, hist = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
            df['macd_histogram'] = hist
            df['stoch_k'], _ = talib.STOCH(high, low, close, fastk_period=14, slowk_period=3, slowd_period=3)
            df['williams_r'] = talib.WILLR(high, low, close, timeperiod=14)
            df['roc_10'] = talib.ROC(close, timeperiod=10)
            return df
        
        self.data = self._apply_by_ticker(calc)
        self.finished_features += ['rsi_14', 'macd_histogram', 'stoch_k', 'williams_r', 'roc_10']
    
    def volatility_features(self):
        """4 volatility & range features"""
        def calc(df):
            close, high, low = df['close'], df['high'], df['low']
            
            # Bollinger Bands
            upper, middle, lower = talib.BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2)
            df['bb_position'] = (close - lower) / (upper - lower)
            df['bb_width'] = (upper - lower) / middle
            
            # Parkinson volatility (high-low based)
            df['parkinson_vol'] = np.sqrt((1 / (4 * np.log(2))) * (np.log(high / low) ** 2)).rolling(20).mean()
            
            # Short-term volatility
            df['returns_std_5d'] = df['log_return'].rolling(5).std()
            return df
        
        self.data = self._apply_by_ticker(calc)
        self.finished_features += ['bb_position', 'bb_width', 'parkinson_vol', 'returns_std_5d']
    
    def volume_features(self):
        """3 volume indicators"""
        def calc(df):
            close, volume = df['close'], df['volume']
            
            vol_sma20 = talib.SMA(volume, timeperiod=20)
            df['volume_ratio'] = volume / vol_sma20
            df['obv'] = talib.OBV(close, volume)
            df['volume_zscore'] = (volume - volume.rolling(20).mean()) / volume.rolling(20).std()
            return df
        
        self.data = self._apply_by_ticker(calc)
        self.finished_features += ['volume_ratio', 'obv', 'volume_zscore']
    
    def lagged_features(self):
        """6 lagged features (1-day lag to avoid look-ahead bias)"""
        lag_cols = ['log_return', 'rsi_14', 'volume_ratio', 'macd_histogram', 'bb_position', 'atr_14']
        
        def calc(df):
            for col in lag_cols:
                df[f'{col}_lag1'] = df[col].shift(1)
            return df
        
        self.data = self._apply_by_ticker(calc)
        self.finished_features += [f'{col}_lag1' for col in lag_cols]
    
    def target_variable(self):
        """Target: next day's return (shifted to avoid look-ahead bias)"""
        def calc(df):
            df['target'] = df['log_return'].shift(-1)  # Predict tomorrow's return
            return df
        self.data = self._apply_by_ticker(calc)

# Run the feature engineering pipeline
fe = FeatureEngineer(data)
features_data = fe.run_pipeline()

print(f"Features created: {len(fe.finished_features)}")
print(f"Data shape: {features_data.shape}")
features_data.groupby(level='ticker').head(3)

Features created: 41
Data shape: (15090, 47)


open        high         low       close     volume  \
ticker date                                                                    
AAPL   2015-01-02   24.694237   24.705322   23.798602   24.237553  212818400   
       2015-01-05   24.006988   24.086797   23.368517   23.554737  257142000   
       2015-01-06   23.619027   23.816332   23.195595   23.556953  263188400   
AMZN   2015-01-02   15.629000   15.737500   15.348000   15.426000   55664000   
       2015-01-05   15.350500   15.419000   15.042500   15.109500   55484000   
       2015-01-06   15.112000   15.150000   14.619000   14.764500   70380000   
GOOGL  2015-01-02   26.430301   26.589102   26.196070   26.278946   26480000   
       2015-01-05   26.159844   26.201529   25.693369   25.778227   41182000   
       2015-01-06   25.829833   25.865066   25.087939   25.142031   54456000   
META   2015-01-02   78.034888   78.382458   77.160987   77.905785   18177500   
       2015-01-05   77.439077   78.700264   76.326844   76.654556   26452200   
       2015-01-06   76.694252   77.051748   74.837222   75.621742   27399300   
NFLX   2015-01-02    4.915143    5.033143    4.873143    4.984857  134750000   
       2015-01-05    4.925857    4.925857    4.714714    4.731143  181650000   
       2015-01-06    4.734714    4.764000    4.566143    4.650143  160377000   
SPY    2015-01-02  171.378492  171.793694  169.551596  170.589600  121465900   
       2015-01-05  169.543334  169.709412  167.201605  167.508850  169632600   
       2015-01-06  167.816019  168.339177  165.133823  165.931015  209151400   

                   log_return  overnight_return  intraday_return  \
ticker date                                                        
AAPL   2015-01-02         NaN               NaN        -0.018667   
       2015-01-05   -0.028576         -0.009558        -0.019018   
       2015-01-06    0.000094          0.002726        -0.002632   
AMZN   2015-01-02         NaN               NaN        -0.013074   
       2015-01-05   -0.020731         -0.004906        -0.015824   
       2015-01-06   -0.023098          0.000165        -0.023264   
GOOGL  2015-01-02         NaN               NaN        -0.005743   
       2015-01-05   -0.019238         -0.004543        -0.014695   
       2015-01-06   -0.024989          0.002000        -0.026989   
META   2015-01-02         NaN               NaN        -0.001656   
       2015-01-05   -0.016191         -0.006009        -0.010182   
       2015-01-06   -0.013565          0.000518        -0.014083   
NFLX   2015-01-02         NaN               NaN         0.014084   
       2015-01-05   -0.052238         -0.011906        -0.040331   
       2015-01-06   -0.017269          0.000755        -0.018023   
SPY    2015-01-02         NaN               NaN        -0.004614   
       2015-01-05   -0.018224         -0.006152        -0.012072   
       2015-01-06   -0.009464          0.001832        -0.011296   

                   volatility_20d  atr_14  ...  volume_ratio          obv  \
ticker date                                ...                              
AAPL   2015-01-02             NaN     NaN  ...           NaN  212818400.0   
       2015-01-05             NaN     NaN  ...           NaN  -44323600.0   
       2015-01-06             NaN     NaN  ...           NaN  218864800.0   
AMZN   2015-01-02             NaN     NaN  ...           NaN   55664000.0   
       2015-01-05             NaN     NaN  ...           NaN     180000.0   
       2015-01-06             NaN     NaN  ...           NaN  -70200000.0   
GOOGL  2015-01-02             NaN     NaN  ...           NaN   26480000.0   
       2015-01-05             NaN     NaN  ...           NaN  -14702000.0   
       2015-01-06             NaN     NaN  ...           NaN  -69158000.0   
META   2015-01-02             NaN     NaN  ...           NaN   18177500.0   
       2015-01-05             NaN     NaN  ...           NaN   -8274700.0   
       2015-01-06             NaN     NaN  ...           NaN  

In [91]:
class RuleBasedModel:
    def __init__(self, data):
        self.data = data.copy()
        self.binary_features = [
            'golden_cross', 'short_uptrend', 'price_above_sma20', 'price_above_sma50',
            'rsi_oversold', 'rsi_overbought', 'macd_bullish', 'roc_positive', 'stoch_oversold',
            'bb_oversold', 'bb_overbought', 'high_volatility',
            'volume_spike', 'volume_confirmation', 'volume_divergence'
        ]
        # Default weights: positive = bullish signal, negative = bearish signal
        self.weights = {
            # Trend (bullish)
            'golden_cross': 2.0,
            'short_uptrend': 1.5,
            'price_above_sma20': 1.0,
            'price_above_sma50': 1.0,
            # Momentum
            'rsi_oversold': 1.5,        # Oversold = buy opportunity
            'rsi_overbought': -1.5,     # Overbought = sell signal
            'macd_bullish': 1.5,
            'roc_positive': 1.0,
            'stoch_oversold': 1.0,
            # Volatility/Reversion
            'bb_oversold': 1.5,         # Mean reversion buy
            'bb_overbought': -1.5,      # Mean reversion sell
            'high_volatility': -0.5,    # High vol = reduce exposure
            # Volume
            'volume_spike': 0.5,
            'volume_confirmation': 1.5, # Price up + volume = strong
            'volume_divergence': -1.5   # Price down + volume = weak
        }
        self.results = None
    
    def run_pipeline(self):
        self._calculate_scores()
        self._generate_signals()
        self._evaluate_performance()
    
    def _apply_by_ticker(self, func):
        return self.data.groupby(level='ticker', group_keys=False).apply(func)
    
    def _calculate_scores(self):
        """Calculate weighted score from binary features"""
        self.data['rule_score'] = sum(
            self.data[feat] * self.weights[feat] for feat in self.binary_features
        )
        # Normalize to [-1, 1] range
        max_pos = sum(w for w in self.weights.values() if w > 0)
        max_neg = abs(sum(w for w in self.weights.values() if w < 0))
        self.data['rule_score_norm'] = self.data['rule_score'].apply(
            lambda x: x / max_pos if x > 0 else x / max_neg if x < 0 else 0
        )
    
    def _generate_signals(self):
        """Convert scores to trading signals: 1=long, 0=neutral, -1=short"""
        # Thresholds for signal generation
        long_threshold = 0.2
        short_threshold = -0.2
        
        self.data['signal'] = 0
        self.data.loc[self.data['rule_score_norm'] > long_threshold, 'signal'] = 1
        self.data.loc[self.data['rule_score_norm'] < short_threshold, 'signal'] = -1
        
        # Strategy return: shift signal to simulate real trading (signal today → trade tomorrow)
        self.data['strategy_return'] = self.data['signal'].shift(1) * self.data['log_return']
    
    def _evaluate_performance(self):
        """Calculate performance metrics per ticker and overall"""
        def calc_metrics(df):
            df = df.dropna(subset=['strategy_return', 'log_return'])
            if len(df) == 0:
                return pd.Series()
            
            strat_ret = df['strategy_return']
            buy_hold = df['log_return']  # Use actual returns, not shifted target
            
            # Annualized metrics
            trading_days = 252
            
            # Cumulative returns - FIXED: Use exp(sum) for log returns
            strat_cum = np.exp(strat_ret.sum()) - 1
            bh_cum = np.exp(buy_hold.sum()) - 1
            
            # Sharpe ratio (annualized)
            sharpe = (strat_ret.mean() / strat_ret.std()) * np.sqrt(trading_days) if strat_ret.std() > 0 else 0
            
            # Max drawdown - FIXED: Use log space
            cum_log_rets = strat_ret.cumsum()
            running_max = cum_log_rets.expanding().max()
            drawdown = cum_log_rets - running_max
            max_dd = np.exp(drawdown.min()) - 1
            
            # Win rate
            wins = (strat_ret > 0).sum()
            total_trades = (df['signal'] != 0).sum()
            win_rate = wins / total_trades if total_trades > 0 else 0
            
            return pd.Series({
                'total_return': strat_cum,
                'buy_hold_return': bh_cum,
                'sharpe_ratio': sharpe,
                'max_drawdown': max_dd,
                'win_rate': win_rate,
                'n_trades': total_trades,
                'n_days': len(df)
            })
        
        # Per-ticker results
        self.results = self.data.groupby(level='ticker').apply(calc_metrics)
        
        # Print summary
        print("=" * 60)
        print("RULE-BASED MODEL PERFORMANCE")
        print("=" * 60)
        for ticker in self.results.index:
            r = self.results.loc[ticker]
            print(f"\n{ticker}:")
            print(f"  Strategy Return: {r['total_return']*100:>8.2f}%  |  Buy & Hold: {r['buy_hold_return']*100:>8.2f}%")
            print(f"  Sharpe Ratio:    {r['sharpe_ratio']:>8.2f}   |  Max Drawdown: {r['max_drawdown']*100:>7.2f}%")
            print(f"  Win Rate:        {r['win_rate']*100:>8.1f}%  |  Trades: {int(r['n_trades'])}")
        
        # Overall
        overall = self.data.dropna(subset=['strategy_return'])
        overall_sharpe = (overall['strategy_return'].mean() / overall['strategy_return'].std()) * np.sqrt(252)
        print(f"\n{'='*60}")
        print(f"OVERALL SHARPE: {overall_sharpe:.3f}")



In [92]:

print("start")

# Get feature columns (exclude target and original OHLCV columns)
feature_cols = fe.finished_features
model_df = features_data.dropna(subset=feature_cols + ['target']).copy()

# CRITICAL FIX: Split by DATE, not by row index (avoid mixing tickers across time)
dates = model_df.index.get_level_values('date').unique().sort_values()
train_end = dates[int(len(dates) * 0.8)]  # 80/20 split

train_mask = model_df.index.get_level_values('date') <= train_end
test_mask = model_df.index.get_level_values('date') > train_end

P_train = model_df.loc[train_mask, feature_cols]
P_test = model_df.loc[test_mask, feature_cols]
H_train = model_df.loc[train_mask, 'target']
H_test = model_df.loc[test_mask, 'target']

print(f"\nFeature set: {len(feature_cols)} features")
print(f"Training: {len(P_train)} samples, Test: {len(P_test)} samples")
print(f"Target mean (train): {H_train.mean():.6f}, std: {H_train.std():.6f}")

# Train XGBoost Regressor with simpler, more robust hyperparameters
model = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    objective='reg:squarederror'
)

model.fit(P_train, H_train)
preds = model.predict(P_test)

# DEBUGGING: Check model predictions
print("\n" + "="*80)
print("MODEL DIAGNOSTICS")
print("="*80)
print(f"Prediction Statistics:")
print(f"  Mean: {preds.mean():.6f}")
print(f"  Std:  {preds.std():.6f}")
print(f"  Min:  {preds.min():.6f}")
print(f"  Max:  {preds.max():.6f}")
print(f"  Unique values: {len(np.unique(preds))}")
if preds.std() < 0.0001:
    print(f"\n  ⚠️  CRITICAL: Model is predicting constant values!")
    print(f"  This indicates the model failed to learn.")
else:
    print(f"\n  ✓ Model is generating varied predictions")
print("="*80 + "\n")

# Generate signals on test data
test_data = model_df[test_mask].copy()
test_data['xgb_pred'] = preds

# Signal generation with reasonable threshold
threshold = 0.0005  # 0.05% daily return threshold
test_data['xgb_signal'] = 0
test_data.loc[test_data['xgb_pred'] > threshold, 'xgb_signal'] = 1
test_data.loc[test_data['xgb_pred'] < -threshold, 'xgb_signal'] = -1

print(f"Signal Distribution (threshold = {threshold}):")
signal_counts = test_data['xgb_signal'].value_counts().sort_index()
for sig in [-1, 0, 1]:
    count = signal_counts.get(sig, 0)
    pct = count / len(test_data) * 100
    label = {-1: 'Short', 0: 'Neutral', 1: 'Long'}[sig]
    print(f"  {label:>7} ({sig:>2}): {count:>5} days ({pct:>5.1f}%)")
print()

# Shift signal forward to simulate real trading: signal today → trade tomorrow
test_data['xgb_return'] = test_data['xgb_signal'].shift(1) * test_data['log_return']


rb_test = RuleBasedModel(test_data)
rb_test._calculate_scores()
rb_test._generate_signals()
test_data['rb_signal'] = rb_test.data['signal']
test_data['rb_return'] = rb_test.data['strategy_return']


def calculate_metrics(returns_col, signal_col, df):
    results = {}
    for ticker in df.index.get_level_values('ticker').unique():
        ticker_data = df.xs(ticker, level='ticker').dropna(subset=[returns_col, 'log_return'])
        if len(ticker_data) == 0:
            continue
        
        strat_ret = ticker_data[returns_col]
        buy_hold = ticker_data['log_return']  # Use actual returns, not shifted target
        signals = ticker_data[signal_col]

        # CRITICAL FIX: Use exp(sum(log_returns)) for proper compounding
        strat_cum = np.exp(strat_ret.sum()) - 1
        bh_cum = np.exp(buy_hold.sum()) - 1
        sharpe = (strat_ret.mean() / strat_ret.std()) * np.sqrt(252) if strat_ret.std() > 0 else 0
        
        # Max drawdown using log returns
        cum_log_rets = strat_ret.cumsum()
        running_max = cum_log_rets.expanding().max()
        drawdown = cum_log_rets - running_max
        max_dd = np.exp(drawdown.min()) - 1
        
        wins = (strat_ret > 0).sum()
        trades = (signals != 0).sum()
        win_rate = wins / trades if trades > 0 else 0

        results[ticker] = {
            'return': strat_cum,
            'bh_return': bh_cum,
            'sharpe': sharpe,
            'max_dd': max_dd,
            'win_rate': win_rate,
            'trades': trades
        }

    all_returns = df.dropna(subset=[returns_col])[returns_col]
    overall_sharpe = (all_returns.mean() / all_returns.std()) * np.sqrt(252) if all_returns.std() > 0 else 0

    return results, overall_sharpe


xgb_results, xgb_overall = calculate_metrics('xgb_return', 'xgb_signal', test_data)
rb_results, rb_overall = calculate_metrics('rb_return', 'rb_signal', test_data)


print("="*80)
print("MODEL COMPARISON - TEST PERIOD ONLY")
print("="*80)
print(f"Period: {dates[int(len(dates)*0.8)+1].date()} to {dates[-1].date()}")
print(f"Test days: {len(dates) - int(len(dates)*0.8)}")
print("="*80)

for ticker in sorted(xgb_results.keys()):
    xgb_metrics = xgb_results[ticker]
    rb_metrics = rb_results[ticker]

    print(f"\n{ticker}:")
    print(f"  {'Metric':<20} {'XGBoost':>12} {'Rule-Based':>12} {'Difference':>12}")
    print(f"  {'-'*20} {'-'*12} {'-'*12} {'-'*12}")
    print(f"  {'Strategy Return':<20} {xgb_metrics['return']*100:>11.2f}% {rb_metrics['return']*100:>11.2f}% {(xgb_metrics['return']-rb_metrics['return'])*100:>11.2f}%")
    print(f"  {'Buy & Hold':<20} {xgb_metrics['bh_return']*100:>11.2f}% {rb_metrics['bh_return']*100:>11.2f}%")
    print(f"  {'Sharpe Ratio':<20} {xgb_metrics['sharpe']:>12.2f} {rb_metrics['sharpe']:>12.2f} {xgb_metrics['sharpe']-rb_metrics['sharpe']:>12.2f}")
    print(f"  {'Max Drawdown':<20} {xgb_metrics['max_dd']*100:>11.2f}% {rb_metrics['max_dd']*100:>11.2f}% {(xgb_metrics['max_dd']-rb_metrics['max_dd'])*100:>11.2f}%")
    print(f"  {'Win Rate':<20} {xgb_metrics['win_rate']*100:>11.1f}% {rb_metrics['win_rate']*100:>11.1f}% {(xgb_metrics['win_rate']-rb_metrics['win_rate'])*100:>11.1f}%")
    print(f"  {'Trades':<20} {xgb_metrics['trades']:>12} {rb_metrics['trades']:>12} {xgb_metrics['trades']-rb_metrics['trades']:>12}")

print(f"\n{'='*80}")
print(f"OVERALL SHARPE RATIO:")
print(f"  XGBoost:    {xgb_overall:>6.3f}")
print(f"  Rule-Based: {rb_overall:>6.3f}")
print(f"  Difference: {xgb_overall - rb_overall:>6.3f}")
print("="*80)

start

Feature set: 41 features
Training: 11838 samples, Test: 2952 samples
Target mean (train): 0.000631, std: 0.021154

MODEL DIAGNOSTICS
Prediction Statistics:
  Mean: -0.000054
  Std:  0.004704
  Min:  -0.143414
  Max:  0.037158
  Unique values: 1922

  ✓ Model is generating varied predictions

Signal Distribution (threshold = 0.0005):
    Short (-1):   672 days ( 22.8%)
  Neutral ( 0):   839 days ( 28.4%)
     Long ( 1):  1441 days ( 48.8%)

MODEL COMPARISON - TEST PERIOD ONLY
Period: 2023-01-13 to 2024-12-27
Test days: 493

AAPL:
  Metric                    XGBoost   Rule-Based   Difference
  -------------------- ------------ ------------ ------------
  Strategy Return            39.84%       86.07%      -46.23%
  Buy & Hold                 91.66%       91.66%
  Sharpe Ratio                 0.93         1.58        -0.65
  Max Drawdown              -23.99%      -15.20%       -8.78%
  Win Rate                    55.0%        57.1%        -2.2%
  Trades                        351  